In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
import seaborn as sns

# import data 
df = pd.read_csv("data/sqli.csv", encoding='utf-16')
print("Data Shape:", df.shape)

Data Shape: (4200, 2)


In [2]:
X = df['Sentence']
y = df['Label']
print(X.shape, y.shape)
len(X)
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

(4200,) (4200,)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### CNN Model

In [13]:
from tensorflow.keras.models import model_from_json

json_file = open('cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn_model = model_from_json(loaded_model_json)
# load weights into new model
cnn_model.load_weights("cnn_model.h5")
print("Loaded model from disk")

# # Use the loaded model to make predictions
# cnn_model.predict(x_inp.reshape(-1,1,4717)).flatten()

Loaded model from disk


### RL model

In [14]:
import numpy as np
import gym

#### CREATING MODEL, AGENT & ENVIRONMENT

In [15]:
class QLearningBinaryClassifier:
    def __init__(self, num_features, learning_rate, discount_factor):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.weights = np.zeros(num_features)

    def predict(self, state):
        q_value = np.dot(self.weights, state)
        action = 1 if q_value >= 0 else 0
        return action

    def update(self, state, action, reward, next_state):
        q_value = np.dot(self.weights, state)
        next_q_value = np.dot(self.weights, next_state)
        target = reward + self.discount_factor * next_q_value
        error = target - q_value
        self.weights += self.learning_rate * error * state

In [16]:
# Define the RL agent (QLearining model )
class SQLInjectionAgent:
    def __init__(self):
        self.model = QLearningBinaryClassifier(num_features=5, learning_rate = 0.1, discount_factor = 0.9)  # Define the RL model (e.g., Q-learning, SARSA)

    def act(self, state):
        action = self.model.predict(state)  # Use the model to predict the best action
        return action

In [17]:
# apriori
def is_injection(xi):
    # x_inp = X[random.randint(0,len(X)-1)]
    return cnn_model.predict(xi.reshape(-1,1,4717)).flatten()

# Define the RL environment (QLearining model )

class SQLInjectionEnv(gym.Env):
    
    def __init__(self):
        self.state = None
        self.done = False

    def reset(self):
        # Generate a new random SQL query to use as the initial state
        # self.state = generate_random_query()
        self.done = False
        # return self.state

    def step(self, state, action):
        # Execute the query with the given action (e.g., input, modification)
        # query = apply_action_to_query(self.state, action)
        # self.index = self.index+1
        query = state
        # query = X_train[self.index]
        result = is_injection(query)

        # Determine the reward based on the result of the query
        if is_injection(query):
            reward = -1  # Penalty for a successful injection
            self.done = True
        elif result == 'error':
            reward = -0.5  # Penalty for a failed query
        else:
            reward = 0.5  # Reward for a successful query

        return query, reward, self.done, {}


#### TTRAINING THE MODEL

In [18]:
# Train the RL agent

env = SQLInjectionEnv()
agent = SQLInjectionAgent()

num_features = 4717 # featues of data
learning_rate = 0.1
discount_factor = 0.9

q_model = QLearningBinaryClassifier(num_features, learning_rate, discount_factor)

out = []

num_episodes = len(X_train)-1

for state in X_train:
    # state = env.reset() # Initialize the state
    done = False
    while not done:
        action = q_model.predict(state)
        out.append(action)
        next_state, reward, done, _ = env.step(state,action) # Observe the next state, reward, and done flag
        q_model.update(state, action, reward, next_state)
        state = next_state


1/1 [==============================] - 0s 36ms/step


#### TESTING & ACCURACY

In [19]:
half = len(X_test)//2
X_test_1 = X_test[:half]
X_val_1 = X_test[half:]
Y_test_1 = y_test[:half]
Y_val_1 = y_test[half:]

In [20]:
# prediction while model not learning

y_pred1 = []
for state in X_test_1:
    action = q_model.predict(state)
    y_pred1.append(action)

In [21]:
accuracy = accuracy_score(Y_test_1, y_pred1)
print("Accuracy:", accuracy)


Accuracy: 0.7761904761904762


In [22]:
# prediction while model learns

y_pred = []

for state in X_test_1:
    # state = env.reset() # Initialize the state
    done = False
    while not done:
        action = q_model.predict(state)
        y_pred.append(action)
        next_state, reward, done, _ = env.step(state,action) # Observe the next state, reward, and done flag
        q_model.update(state, action, reward, next_state)
        state = next_state

1/1 [==============================] - 0s 22ms/step


In [24]:
accuracy = accuracy_score(Y_test_1, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7761904761904762


#### VALIDATION & ACCURACY

In [25]:
# prediction while model not learning

y_pred2 = []
for state in X_val_1:
    action = q_model.predict(state)
    y_pred2.append(action)

In [26]:
accuracy = accuracy_score(Y_val_1, y_pred2)
print("Accuracy:", accuracy)


Accuracy: 0.7904761904761904


In [27]:
# prediction while model learns

y_pred3 = []

for state in X_val_1:
    # state = env.reset() # Initialize the state
    done = False
    while not done:
        action = q_model.predict(state)
        y_pred3.append(action)
        next_state, reward, done, _ = env.step(state,action) # Observe the next state, reward, and done flag
        q_model.update(state, action, reward, next_state)
        state = next_state

1/1 [==============================] - 0s 20ms/step


In [28]:
accuracy = accuracy_score(Y_val_1, y_pred3)
print("Accuracy:", accuracy)


Accuracy: 0.7880952380952381
